# Movielens 영화 추천 실습

-MovieLens 1M Dataset 사용을 권장   
-별점 데이터는 대표적인 explicit 데이터이지만 implicit 데이터로 간주하고 테스트할 수 있다.   
-별점을 시청횟수로 해석   
-유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정, 제외

## 1. 데이터 준비 및 전처리

In [534]:
import pandas as pd
import os
from pandas import DataFrame
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [535]:
# 3점 이상만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [536]:
# rating 컬럼의 이름을 count로 바꾼다
ratings.rename(columns={'rating':'count'}, inplace=True)

In [537]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [538]:
# 영화 제목을 보기 위한 메타 데이터 불러오기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies[1:60]

,movie_id,title,genre
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller
10,11,"American President, The (1995)",Comedy|Drama|Romance


In [539]:
data3 = pd.merge(ratings, movies, on='movie_id')

In [540]:
del data3['genre']
data3

,user_id,movie_id,count,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975)
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975)
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975)
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975)
...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973)
836474,5854,3026,4,958346883,Slaughterhouse (1987)
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)"
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)"


df_13_axis1 = pd.concat([ratings, movies], axis=1)

del df_13_axis1['timestamp']
df_13_axis1

## 2. 분석

In [541]:
# 유니크한 영화 수
ratings['movie_id'].nunique()

3628

In [542]:
# 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [543]:
# 인기 많은 영화 30
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

## 3. 선호하는 영화 5가지를 rating에 추가

In [544]:
my_movie_num = ['13' , '14' ,'15' ,'16' ,'17']

my_favorite = ["Balto (1995)", "Nixon (1995)", "Cutthroat Island (1995)", "Casino (1995)", "Sense and Sensibility (1995)"]

my_playlist = pd.DataFrame({'user_id': ['jaden']*5, 'movie_id': my_movie_num, 'title' : my_favorite, 'count':[5]*5})

if not data3.isin({'user_id':['jaden']})['user_id'].any():  # user_id에 'jaden'이라는 데이터가 없다면
    rating = data3.append(my_playlist)                          # 위에 임의로 만든 my_favorite 데이터를 추가
del rating['timestamp']
rating.tail(10)    

,user_id,movie_id,count,title
836473,5851,3607,5,One Little Indian (1973)
836474,5854,3026,4,Slaughterhouse (1987)
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)"
836477,5948,1360,5,Identification of a Woman (Identificazione di ...
0,jaden,13,5,Balto (1995)
1,jaden,14,5,Nixon (1995)
2,jaden,15,5,Cutthroat Island (1995)
3,jaden,16,5,Casino (1995)
4,jaden,17,5,Sense and Sensibility (1995)


### 데이터 전처리

In [545]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique =rating['user_id'].unique()
#movie_unique = rating['movie_id'].unique()
movie_unique = rating['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [546]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['jaden'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(movie_to_idx['Balto (1995)'])

6039
1513


In [547]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = rating['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(rating):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    rating['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = rating['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(rating):
    print('title column indexing OK!!')
    rating['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

rating

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,1,1193,5,0
2,2,1193,4,0
3,3,1193,4,0
4,4,1193,5,0
...,...,...,...,...
0,6039,13,5,1513
1,6039,14,5,436
2,6039,15,5,2105
3,6039,16,5,281


## 4. CSR matrix 모델 만들기

In [548]:
rating.tail()

,user_id,movie_id,count,title
0,6039,13,5,1513
1,6039,14,5,436
2,6039,15,5,2105
3,6039,16,5,281
4,6039,17,5,357


In [549]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = rating['user_id'].nunique()
num_movie = rating['title'].nunique()

csr_data = csr_matrix((rating['count'], (rating.user_id, rating.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

rating.user_id

rating['user_id']

## 5. AlternatingLeastSquares 모델 구성 및 훈련

In [550]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

In [627]:
#Implicit AlternatingLeastSquares 모델 선언
als_model = AlternatingLeastSquares(factors=250, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [628]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [629]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

In [630]:
jaden, fourth = user_to_idx['jaden'], movie_to_idx['Balto (1995)']
jaden_vector, fourth_vector = als_model.user_factors[jaden], als_model.item_factors[fourth]

## 6. 선호하는 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

In [631]:
np.dot(jaden_vector, fourth_vector)

0.14140391

In [632]:
first = movie_to_idx['Nixon (1995)']
first_vector = als_model.item_factors[first]
np.dot(jaden_vector, first_vector)

0.2724098

## 7. 내가 좋아하는 영화와 비슷한 영화 추천받기

In [633]:
favorite_movie = 'Nixon (1995)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(436, 1.0000001),
 (3006, 0.6072077),
 (2961, 0.60208964),
 (2699, 0.58935755),
 (2881, 0.5874487),
 (3368, 0.5862714),
 (3435, 0.5849055),
 (1217, 0.5845963),
 (3469, 0.58402026),
 (3541, 0.5828926),
 (2608, 0.58138007),
 (3522, 0.58080375),
 (3523, 0.58053124),
 (396, 0.5798057),
 (2653, 0.5796318)]

In [634]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Nixon (1995)',
 'B*A*P*S (1997)',
 'Rent-A-Cop (1988)',
 'Losing Chase (1996)',
 'Twilight (1998)',
 'Head Above Water (1996)',
 'Woman of Paris, A (1923)',
 'Blue Chips (1994)',
 'Cotton Mary (1999)',
 'Smashing Time (1967)',
 'Wisdom of Crocodiles, The (a.k.a. Immortality) (2000)',
 'Dangerous Ground (1997)',
 'I Got the Hook Up (1998)',
 'Poetic Justice (1993)',
 'Two Bits (1995)']

In [635]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [636]:
get_similar_movie('Nixon (1995)')

['Nixon (1995)',
 'B*A*P*S (1997)',
 'Rent-A-Cop (1988)',
 'Losing Chase (1996)',
 'Twilight (1998)',
 'Head Above Water (1996)',
 'Woman of Paris, A (1923)',
 'Blue Chips (1994)',
 'Cotton Mary (1999)',
 'Smashing Time (1967)']

In [637]:
user = user_to_idx['jaden']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(419, 0.22322184),
 (352, 0.21269566),
 (407, 0.20296827),
 (101, 0.16983),
 (146, 0.1591441),
 (53, 0.15579072),
 (840, 0.15417863),
 (381, 0.15098308),
 (204, 0.14853515),
 (566, 0.14811605),
 (76, 0.14423938),
 (98, 0.14081104),
 (809, 0.13884614),
 (400, 0.13849163),
 (406, 0.1370739),
 (474, 0.13370627),
 (972, 0.12911463),
 (449, 0.1285348),
 (72, 0.12538436),
 (1004, 0.12530385)]

In [638]:
[idx_to_movie[i[0]] for i in movie_recommended]

['JFK (1991)',
 'Emma (1996)',
 'Crimson Tide (1995)',
 'Remains of the Day, The (1993)',
 'Thelma & Louise (1991)',
 'Shine (1996)',
 'Simple Plan, A (1998)',
 'Elizabeth (1998)',
 'Clueless (1995)',
 "Breakfast at Tiffany's (1961)",
 'Outbreak (1995)',
 'Mis�rables, Les (1998)',
 'Bulworth (1998)',
 'Postino, Il (The Postman) (1994)',
 'People vs. Larry Flynt, The (1996)',
 'Chicken Run (2000)',
 'Mars Attacks! (1996)',
 'Leaving Las Vegas (1995)',
 'Fisher King, The (1991)',
 'Frankenstein (1931)']